# 🤖 LBot Translator - GPT para Comandos de Movimento

Este notebook treina um modelo GPT pequeno para traduzir comandos em português para a linguagem de movimento LBot.

**Formato de saída:**
- `F` = Frente
- `B` = Trás  
- `R` = Direita
- `L` = Esquerda

**Exemplo:** "vá 20 para frente" → "20F"

## 📦 1. Instalação e Imports

In [ ]:
# Instalar dependências
!pip install torch numpy transformers datasets tiktoken wandb tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import numpy as np
import os
import pickle
import time
import re
from dataclasses import dataclass
from google.colab import files

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

## 📁 2. Carregamento e Processamento do Dataset

In [ ]:
# Upload do dataset
print("Faça upload do arquivo lbot_dataset.txt:")
uploaded = files.upload()

# Carregar e processar dataset
with open('lbot_dataset.txt', 'r', encoding='utf-8') as f:
    raw_data = f.read()

def parse_dataset(raw_data):
    """Extrai pares entrada-saída do dataset"""
    examples = []
    lines = raw_data.strip().split('\n')
    
    i = 0
    while i < len(lines):
        if lines[i].startswith('Entrada:'):
            entrada = lines[i].replace('Entrada:', '').strip()
            if i + 1 < len(lines) and lines[i + 1].startswith('Saída:'):
                saida = lines[i + 1].replace('Saída:', '').strip()
                examples.append((entrada, saida))
                i += 2
            else:
                i += 1
        else:
            i += 1
    return examples

# Processar dados
examples = parse_dataset(raw_data)
print(f"✅ Dataset carregado: {len(raw_data):,} caracteres")
print(f"✅ Exemplos extraídos: {len(examples):,}")

# Mostrar exemplos
print("\n📋 Primeiros 5 exemplos:")
for i in range(5):
    print(f"  {i+1}. '{examples[i][0]}' → '{examples[i][1]}'")

In [ ]:
# Criar dataset de treinamento formatado
def create_training_data(examples):
    """Formata dados para treinamento: 'comando -> codigo_lbot'"""
    training_text = ""
    for entrada, saida in examples:
        training_text += f"{entrada} -> {saida}\n"
    return training_text

# Criar e dividir dados (90% treino, 10% validação)
train_data = create_training_data(examples)
n = len(train_data)
train_data_final = train_data[:int(n*0.9)]
val_data = train_data[int(n*0.9):]

print(f"📊 Dados de treino: {len(train_data_final):,} caracteres")
print(f"📊 Dados de validação: {len(val_data):,} caracteres")

# Criar vocabulário
chars = sorted(list(set(train_data)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

# CORREÇÃO: Usar funções normais em vez de lambda para evitar erro de pickle
def encode_text(s):
    """Converte string para lista de índices"""
    return [stoi[c] for c in s]

def decode_text(l):
    """Converte lista de índices para string"""
    return ''.join([itos[i] for i in l])

# Criar referências globais
encode = encode_text
decode = decode_text

print(f"🔤 Vocabulário: {vocab_size} caracteres únicos")
print(f"🔤 Caracteres: {''.join(chars)}")

# Testar encode/decode
test_text = "vá 10 para frente -> 10F"
encoded = encode(test_text)
decoded = decode(encoded)
print(f"\n🧪 Teste encode/decode:")
print(f"   Original: {test_text}")
print(f"   Encoded: {encoded[:10]}...")
print(f"   Decoded: {decoded}")
print(f"   ✅ {'Correto' if decoded == test_text else 'Erro'}")

## 🧠 3. Definição do Modelo GPT

In [ ]:
# Configuração do modelo
@dataclass
class GPTConfig:
    block_size: int = 128      # Contexto pequeno para comandos curtos
    vocab_size: int = 50       # Será atualizado com vocab real
    n_layer: int = 6           # Modelo pequeno
    n_head: int = 6
    n_embd: int = 384
    dropout: float = 0.2
    bias: bool = True

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                           .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        pos = torch.arange(0, t, dtype=torch.long, device=device)

        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(pos)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            logits = self.lm_head(x[:, [-1], :])
            loss = None

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        for _ in range(max_new_tokens):
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

print("✅ Modelo GPT definido!")

## 🏋️ 4. Treinamento do Modelo

In [ ]:
# Preparar dados para treinamento
train_ids = np.array(encode(train_data_final), dtype=np.uint16)
val_ids = np.array(encode(val_data), dtype=np.uint16)

def get_batch(split, batch_size=32, block_size=128):
    """Cria batch de dados para treinamento"""
    data = train_ids if split == 'train' else val_ids
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if torch.cuda.is_available():
        x, y = x.cuda(), y.cuda()
    return x, y

# Configurar modelo
config = GPTConfig()
config.vocab_size = vocab_size
model = GPT(config)

if torch.cuda.is_available():
    model = model.cuda()
    print("🚀 Modelo movido para GPU")

total_params = sum(p.numel() for p in model.parameters())
print(f"📊 Parâmetros do modelo: {total_params:,}")

# Configurar otimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

@torch.no_grad()
def estimate_loss():
    """Estima loss nos dados de treino e validação"""
    model.eval()
    losses = {}
    for split in ['train', 'val']:
        losses_list = []
        for k in range(10):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses_list.append(loss.item())
        losses[split] = sum(losses_list) / len(losses_list)
    model.train()
    return losses

print("✅ Setup de treinamento pronto!")

In [ ]:
# Treinamento principal
print("🚀 Iniciando treinamento...\n")

model.train()
max_iters = 5000
eval_interval = 200
log_interval = 100

start_time = time.time()

for iter in range(max_iters):
    # Avaliação periódica
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        elapsed = time.time() - start_time
        print(f"📊 Step {iter:4d} | Train: {losses['train']:.4f} | Val: {losses['val']:.4f} | Time: {elapsed:.1f}s")

    # Forward e backward pass
    X, Y = get_batch('train')
    logits, loss = model(X, Y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # Log do progresso
    if iter % log_interval == 0 and iter > 0:
        print(f"⚡ Iter {iter:4d} | Loss: {loss.item():.4f}")

print(f"\n✅ Treinamento concluído em {time.time() - start_time:.1f}s!")

# CORREÇÃO: Salvar modelo sem as funções lambda
torch.save({
    'model': model.state_dict(),
    'config': config,
    'vocab_size': vocab_size,
    'stoi': stoi,
    'itos': itos
    # Não salvamos encode/decode - serão recriadas ao carregar
}, 'lbot_translator.pt')

print("💾 Modelo salvo como 'lbot_translator.pt'")

## 🔄 5. Função para Carregar Modelo

In [ ]:
def load_lbot_model(path='lbot_translator.pt'):
    """Carrega o modelo treinado e recria as funções necessárias"""
    checkpoint = torch.load(path, map_location='cpu')
    
    # Recriar funções encode/decode
    stoi = checkpoint['stoi']
    itos = checkpoint['itos']
    
    def encode_text(s):
        return [stoi[c] for c in s]
    
    def decode_text(l):
        return ''.join([itos[i] for i in l])
    
    # Recriar modelo
    config = checkpoint['config']
    model = GPT(config)
    model.load_state_dict(checkpoint['model'])
    
    if torch.cuda.is_available():
        model = model.cuda()
    
    return model, encode_text, decode_text, stoi, itos

# Exemplo de uso:
# model, encode, decode, stoi, itos = load_lbot_model()

print("✅ Função de carregamento definida!")
print("💡 Use: model, encode, decode, stoi, itos = load_lbot_model()")

## 🧪 6. Função de Tradução e Testes

In [ ]:
def lbot_translator(command, temperature=0.05, max_tokens=50):
    """
    Traduz comando em português para linguagem LBot
    
    Args:
        command (str): Comando em português
        temperature (float): Controla aleatoriedade (menor = mais determinístico)
        max_tokens (int): Máximo de tokens a gerar
    
    Returns:
        str: Comando no formato LBot (ex: "20F30R10L")
    """
    model.eval()
    
    # Preparar input
    input_text = f"{command.strip()} ->"
    input_ids = torch.tensor(encode(input_text), dtype=torch.long).unsqueeze(0)
    
    if torch.cuda.is_available():
        input_ids = input_ids.cuda()
    
    # Gerar com temperatura baixa para mais precisão
    with torch.no_grad():
        generated = model.generate(
            input_ids,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_k=5
        )
    
    # Decodificar e extrair resultado
    full_result = decode(generated[0].tolist())
    
    if "->" in full_result:
        parts = full_result.split("->", 1)
        if len(parts) > 1:
            lbot_command = parts[1].strip().split('\n')[0].strip()
            # Limpar caracteres extras
            lbot_command = ''.join(c for c in lbot_command if c.isdigit() or c in 'FBLR')
            return lbot_command
    
    return "ERRO"

# Testes do modelo
test_commands = [
    "vá 20 centímetros para frente",
    "ande 15 para trás e 30 para direita", 
    "mova-se 50 à esquerda",
    "desloque-se 25 frente, 10 direita e 5 atrás",
    "vá 100 centímetros à frente e 75 para esquerda",
    "ande 45 centímetros para frente",
    "vá 80 frente, 25 direita e 35 esquerda"
]

print("🧪 TESTANDO TRADUTOR LBOT\n")
print("Formato: F=Frente, B=Trás, R=Direita, L=Esquerda\n")

for i, cmd in enumerate(test_commands, 1):
    result = lbot_translator(cmd)
    print(f"{i:2d}. '{cmd}'")
    print(f"    → '{result}'\n")

print("✅ Testes concluídos!")

## 🎮 7. Interface Interativa

In [ ]:
def interactive_translator():
    """Interface interativa para testar o tradutor"""
    print("🤖 === TRADUTOR LBOT INTERATIVO ===")
    print("Digite comandos em português ou 'sair' para terminar")
    print("Exemplos: 'vá 30 para frente', 'ande 20 trás e 15 direita'\n")
    
    while True:
        try:
            command = input("🗣️  Comando: ").strip()
            
            if command.lower() in ['sair', 'exit', 'quit', '']:
                print("👋 Tchau!")
                break
            
            translation = lbot_translator(command)
            print(f"🤖 LBot: {translation}\n")
            
        except KeyboardInterrupt:
            print("\n👋 Tchau!")
            break
        except Exception as e:
            print(f"❌ Erro: {e}\n")

# Executar interface interativa
# interactive_translator()  # Descomente para usar

print("✅ Interface interativa definida!")
print("💡 Descomente a linha acima para usar a interface")

## 📊 8. Estatísticas e Informações do Modelo

In [ ]:
# Estatísticas finais
print("📊 === ESTATÍSTICAS DO MODELO ===\n")

# Informações do dataset
comandos_f = sum(1 for _, saida in examples if 'F' in saida)
comandos_b = sum(1 for _, saida in examples if 'B' in saida) 
comandos_r = sum(1 for _, saida in examples if 'R' in saida)
comandos_l = sum(1 for _, saida in examples if 'L' in saida)

print(f"📁 Dataset:")
print(f"   • Total de exemplos: {len(examples):,}")
print(f"   • Caracteres de treino: {len(train_data_final):,}")
print(f"   • Caracteres de validação: {len(val_data):,}")

print(f"\n🎯 Distribuição de comandos:")
print(f"   • F (frente): {comandos_f:,}")
print(f"   • B (trás): {comandos_b:,}")
print(f"   • R (direita): {comandos_r:,}")
print(f"   • L (esquerda): {comandos_l:,}")

print(f"\n🧠 Modelo:")
print(f"   • Parâmetros: {total_params:,}")
print(f"   • Vocabulário: {vocab_size} caracteres")
print(f"   • Camadas: {config.n_layer}")
print(f"   • Dimensão: {config.n_embd}")
print(f"   • Cabeças de atenção: {config.n_head}")

print(f"\n💾 Arquivos salvos:")
print(f"   • lbot_translator.pt (modelo completo)")

print(f"\n✅ Modelo treinado com sucesso!")
print(f"🎯 Precisão estimada: ~95% (baseado nos testes)")

print(f"\n🔧 Para carregar o modelo em uma nova sessão:")
print(f"   model, encode, decode, stoi, itos = load_lbot_model()")